In [29]:
import collections
import copy
import hashlib
import io
import os
import subprocess
import textwrap
import time

from typing import List, Text

from PIL import Image

import numpy as np
import pandas as pd
# import tabulate


import tensorflow as tf

import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder


In [32]:
# Class to use: sk, lk, df, angk, ha, la, pg, sa, ak, bcc, bd, ch, mpd, scc, dfsp, ks
train_dir = '/Users/francesco/repos/computer_vision_project/dataset/dataverse_files/derm12345_train'
csv_train_dir = '/Users/francesco/repos/computer_vision_project/dataset/dataverse_files/derm12345_metadata_train.csv'
test_dir = '/Users/francesco/repos/computer_vision_project/dataset/dataverse_files/derm12345_test'
csv_test_dir = '/Users/francesco/repos/computer_vision_project/dataset/dataverse_files/derm12345_metadata_test.csv'

df_train = pd.read_csv(csv_train_dir)
df_test = pd.read_csv(csv_test_dir)
df = pd.concat([df_train, df_test], ignore_index=True)


df['id'] = (df['label'] == 'sk')
target_labels = ['sk', 'lk', 'df', 'angk', 'ha', 'la', 'pg', 'sa', 'ak', 'bcc', 'bd', 'ch', 'mpd', 'scc', 'dfsp', 'ks']
df_filtered = df[df['label'].isin(target_labels)]
df_filtered['label'] = 'others'

df_minimal = df_filtered[['image_id', 'label']].copy()
df_minimal['class'] = 'others'
print(len(df_filtered))
print(len(df))
print(df_filtered)


1902
12345
          image_id  patient_id   image_type copyright-license  split  \
607    DERM_172873  PID_152688  dermoscopic             CC-BY  train   
608    DERM_204720  PID_152688  dermoscopic             CC-BY  train   
609    DERM_355404  PID_152688  dermoscopic             CC-BY  train   
610    DERM_263067  PID_233835  dermoscopic             CC-BY  train   
611    DERM_912184  PID_233835  dermoscopic             CC-BY  train   
...            ...         ...          ...               ...    ...   
12324  DERM_175495  PID_996758  dermoscopic             CC-BY   test   
12325  DERM_196069  PID_996758  dermoscopic             CC-BY   test   
12326  DERM_985535  PID_999593  dermoscopic             CC-BY   test   
12327  DERM_985554  PID_999597  dermoscopic             CC-BY   test   
12328  DERM_985555  PID_999597  dermoscopic             CC-BY   test   

          super_class     malignancy   main_class_1   main_class_2  \
607    nonmelanocytic  indeterminate  keratinocytic  k

/var/folders/kk/2c_8134s1jqf2s0cx82ytg3h0000gn/T/ipykernel_13928/683379235.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['label'] = 'others'


In [34]:
X = pd.DataFrame(df_minimal['image_id'].astype('category').cat.codes)  # Convert image_id to numeric
y = df_minimal['class']

target_size = 10000  # Desired size of the augmented dataset
current_size = len(df_minimal)
multiplier = int(np.ceil(target_size / current_size))

augmented_df = pd.concat([df_minimal] * multiplier, ignore_index=True)
augmented_df = augmented_df.head(target_size)  # Trim to exact size if needed

# Add the 'others' class label
augmented_df['class'] = 'others'

print("Original dataset size:", len(df_filtered))
print("Augmented dataset size:", len(augmented_df))
print("\nOriginal unique images:", len(df_minimal['image_id'].unique()))
print("Augmented unique images:", len(augmented_df['image_id'].unique()))

Original dataset size: 1902
Augmented dataset size: 10000

Original unique images: 1902
Augmented unique images: 1902


In [37]:
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os

# Get a sample original image_id
original_image_id = df_minimal['image_id'].iloc[0]

# Construct paths for the image
train_image_path = os.path.join(train_dir, f"{original_image_id}.jpg")
test_image_path = os.path.join(test_dir, f"{original_image_id}.jpg")

# Try both train and test paths since the image could be in either
image_path = train_image_path if os.path.exists(train_image_path) else test_image_path

# Load and display the image
if os.path.exists(image_path):
    # Read image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    
    # Create figure with two subplots
    plt.figure(figsize=(12, 6))
    
    # Plot original image
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')
    
    # In this case, the "augmented" image is the same since we're just duplicating
    plt.subplot(1, 2, 2)
    plt.imshow(image)
    plt.title('Augmented Image (Duplicate)')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print image details
    print(f"Image ID: {original_image_id}")
    print(f"Image Shape: {image.shape}")
else:
    print(f"Image not found at path: {image_path}")

Image not found at path: /Users/francesco/repos/computer_vision_project/dataset/dataverse_files/derm12345_test/DERM_172873.jpg
